In [1]:
import pandas as pd
from collections import defaultdict
from datetime import datetime
from operator import itemgetter
import csv
import os



In [48]:
dpath = os.path.join(os.getcwd(), 'datasets', 'EXTRA')
l = os.listdir(dpath)
names = set([x[24:-7] for x in l])
print(names)
d = {}

for x in names:
    d[x] = pd.DataFrame()
    
for x in l:
    df = pd.read_csv(os.path.join(dpath, x))
    d[x[24:-7]] = d[x[24:-7]].append(df, ignore_index=True)

for x in names:
    d[x].to_csv(os.path.join(os.getcwd(), 'datasets/NSE', '{}.csv'.format(x)), index=False)

{'BPCL', 'INFY', 'LT', 'SBIN', 'M&M'}


In [49]:
for f in names:
    nseName = os.path.join(os.getcwd(), 'datasets/NSE', '{}.csv'.format(f))
    mcName = os.path.join(os.getcwd(), 'datasets/MC', 'MC{}.csv'.format(f))
    nse = pd.read_csv(nseName)
    mc = pd.read_csv(mcName, sep='|')

    # print(nse)
    # print(mc.head())
    nse['Date'] = nse['Date'].apply(lambda x: datetime.strptime(x, '%d-%b-%Y'))
    mc['date'] = mc['date'].apply(lambda x: x.strip()).apply(lambda x: datetime.strptime(x, '%d %b %Y'))
    mc = mc.sort_values(by='date')
    # print(nse.head())
    # print(mc.head())
    
    mydick = defaultdict(list)
    Date = [[row['Date'], row['Close Price']] for i, row in nse.iterrows()]

    for date, text in zip(mc['date'], mc['text']):
        mydick['date'].append(date)
        mydick['text'].append(text)
        myDate = Date[:]
        myDate.append([date, 0])
        myDate.sort(key=itemgetter(0))

        ind = myDate.index([date, 0])
        try:
            prevDay = myDate[ind - 1]
        except IndexError:
            prevDay = myDate[ind + 1]
        try:
            nextDay = myDate[ind + 1]
        except IndexError:
            nextDay = myDate[ind - 1]
        try:
            if prevDay[0] == date:
                prevDay = myDate[ind - 2]
            if nextDay[0] == date:
                nextDay = myDate[ind + 2]
        except IndexError:
            pass
        mydick['prevDay'].append(prevDay[0])
        mydick['prevClose'].append(prevDay[1])
        mydick['nextDay'].append(nextDay[0])
        mydick['nextClose'].append(nextDay[1])
        mydick['Difference'].append(nextDay[1] - prevDay[1])
        
    
    df = pd.DataFrame(mydick)
    df.to_csv(os.path.join(os.getcwd(),'datasets/Merge/MERGE' + f + '.csv'), index=False, sep='|')

    

In [13]:
mydick = defaultdict(list)
Date = [[row['Date'], row['Close Price']] for i, row in nse.iterrows()]

for date, text in zip(mc['date'], mc['text']):
    mydick['date'].append(date)
    mydick['text'].append(text)
    myDate = Date[:]
    myDate.append([date, 0])
    myDate.sort(key=itemgetter(0))

    ind = myDate.index([date, 0])
    try:
        prevDay = myDate[ind - 1]
    except IndexError:
        prevDay = myDate[ind + 1]
    try:
        nextDay = myDate[ind + 1]
    except IndexError:
        nextDay = myDate[ind - 1]
    try:
        if prevDay[0] == date:
            prevDay = myDate[ind - 2]
        if nextDay[0] == date:
            nextDay = myDate[ind + 2]
    except IndexError:
        pass
    mydick['prevDay'].append(prevDay[0])
    mydick['prevClose'].append(prevDay[1])
    mydick['nextDay'].append(nextDay[0])
    mydick['nextClose'].append(nextDay[1])
    mydick['Difference'].append(nextDay[1] - prevDay[1])

# print(mydick['Difference'])


In [17]:
df = pd.DataFrame(mydick)
df.to_csv('Merge/MERGEAIRTEL.csv', index=False, sep='|')
